In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import VGG16, MobileNetV2, InceptionV3, NASNetLarge, NASNetMobile
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet import EfficientNetB3, center_crop_and_resize, preprocess_input

from keras import models
from keras import layers
from keras import callbacks

import time

import matplotlib.pyplot as plt

import cv2

Using TensorFlow backend.


In [2]:
BASEPATH = "stanford-dogs-dataset/"
BATCH_SIZE = 64
EPOCHS = 100

In [3]:
early_stopping = EarlyStopping(patience=20, verbose=1,restore_best_weights=True, monitor="val_acc")
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=7,verbose=1, monitor="val_acc")

In [4]:
def createModelNASNetMobile(trainable=False, optimizer="adam"):
    base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(120, activation="softmax"))
    
    if not trainable:
        for layer in base_model.layers:
            layer.trainable = False

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 
  
    return model

In [5]:
model = createModelNASNetMobile()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Using the ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255.0,
                                   shear_range=0.1,
                                   rotation_range=10.,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=[0.9, 1.1],
                                   brightness_range=[0.8, 1.2],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255.0)

validation_datagen = ImageDataGenerator(rescale=1./255.0)

In [7]:
train_generator = train_datagen.flow_from_directory(
    BASEPATH + "train",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    BASEPATH + "test",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    BASEPATH + "val",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 15394 images belonging to 120 classes.
Found 1114 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [8]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS, callbacks=[early_stopping, reduce_lr])

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
241/241 [==============================] - 246s 1s/step - loss: 1.8795 - acc: 0.5542 - val_loss: 0.6446 - val_acc: 0.8067
Epoch 2/100
241/241 [==============================] - 239s 990ms/step - loss: 1.1508 - acc: 0.6827 - val_loss: 0.6536 - val_acc: 0.7974
Epoch 3/100
241/241 [==============================] - 239s 993ms/step - loss: 1.0748 - acc: 0.6943 - val_loss: 0.6464 - val_acc: 0.8057
Epoch 4/100
241/241 [==============================] - 239s 993ms/step - loss: 1.0130 - acc: 0.7133 - val_loss: 0.6446 - val_acc: 0.8033
Epoch 5/100
241/241 [==============================] - 239s 990ms/step - loss: 0.9643 - acc: 0.7237 - val_loss: 0.6393 - val_acc: 0.8060
Epoch 6/100
241/241 [==============================] - 239s 991ms/step - loss: 0.9215 - acc: 0.7346 - val_loss: 0.6449 - val_acc: 0.8134
Epoch 7/100
241/241 [==============================] - 239s 990ms/step - loss: 0.9176 - acc: 0.7305 - val_loss: 0.6502 - val_acc: 0.8

241/241 [==============================] - 243s 1s/step - loss: 0.6025 - acc: 0.8111 - val_loss: 0.6498 - val_acc: 0.8139
Epoch 00035: early stopping


## Evaluation

In [9]:
loss, acc = model.evaluate_generator(test_generator,verbose=0, steps=test_generator.samples // BATCH_SIZE)

In [10]:
print(loss,acc)

0.6951451136006249 0.8105925


In [14]:
model.save("nasnetlarge-model-{:.2f}acc-{:.2f}loss-{:.0f}.hdf5".format(acc, loss, time.time()))